In [2]:
import numpy as np
import nltk
from collections import defaultdict
from src.classes.database.offer import MongoOffer
from tqdm import tqdm
from enum import Enum

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [3]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
direction = Enum('direction', ['FORWARD', 'BACKWARD', 'SELF'])

In [4]:
def get_all_words(token_list: list[list]) -> list:
    words = []
    for sentence in token_list:
        for word in sentence:
            words.append(word)
    return sorted(set(words))

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def p_next_word(given: str, looking: str, look_ahead: int, tokens: list[list]) -> float:
    nwo: dict = next_word_occurrence(given, look_ahead, tokens)
    return nwo[looking] / sum(nwo.values()) if nwo[looking] != 0 else 0

# Old method, slow and not used anymore
def calculate_all_probabilities(tokens: list[list], look_ahead) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = defaultdict()
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in tqdm(p_word):
        p_word[word] = [defaultdict(float) for x in range(0,look_ahead)]
        for x in range(0,look_ahead):
            for sub_word in all_words:
                p = p_next_word(word, sub_word, x+1, tokens)
                if p > 0:
                    p_word[word][x][sub_word] = p_next_word(word, sub_word, x+1, tokens)
                #print(f"{word} | {str(x)} | {sub_word}: {p_word[word][x][sub_word]}")
    return p_word
#data = calculate_all_probabilities(token_list, 1)

## HMaxtrix
Used for storing all word occurrences in a 3 dimensional matrix

### Data we can get from matrix
 1. p(start, find) = probability that 'find' is a n-order forward word for 'start' | single cell value / sum of 'start' row
 2. p(start, find) = probability that 'find' is a n-order backward word for 'start' | single cell value / sum of 'start' column
 3. p(row_word) = probability that 'row word' will be a n-order forward word | sum of row / entire table sum
 4. p(col_word) = probability that 'col word' will be a n-order backward word | sum of column / entire table sum
 5. p(word, order) = probability that 'word' will be n-order word compared to the other orders| single cell value at order / all order values of that word tallied (through the table)
 6.  p(word-row, order) = probability that 'word' will be n-order forward word | all of word-row order sum / all order values of that word-row
 7. p(col-row, order) = probability that 'word' will be n-order backward word | all of word-col order sum / all order values of that word-col

In [5]:
class HMatrix:
    def __init__(self) -> None:
        self.labels = None
        self.reverse_labels = None
        self.order = 0
        self.matrix = None

    def p_row_word(self, order: int, row_word: str, word: str) -> float:
        if row_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        row_label = self.reverse_labels[row_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, row_label, :][word_label]
        divisor = sum(self.matrix[order, row_label, :])

        return dividend/divisor if divisor > 0 else 0

    def p_col_word(self, order: int, col_word: str, word: str) -> float:
        if col_word not in self.reverse_labels or word not in self.reverse_labels:
            return 0
        col_label = self.reverse_labels[col_word]
        word_label = self.reverse_labels[word]

        dividend = self.matrix[order, :, col_label][word_label]
        divisor = sum(self.matrix[order, :, col_label])

        return dividend/divisor if divisor > 0 else 0


    def create_matrix(self, tokens: list[list], order: int):
        #Setup
        self.labels = get_all_words(token_list)
        self.reverse_labels = {self.labels[x]: x for x in range(0, len(self.labels))}
        self.order = order
        self.matrix = np.zeros((order, len(self.labels), len(self.labels)))

        #Iteration
        for sentence in tqdm(tokens):
            for x in range(0, len(sentence)):
                word = sentence[x]
                print(f"Word: {word} ({str(self.reverse_labels[word])})")
                for y in range(0, self.order):
                    if x+y+1 < len(sentence):
                        print(f"Lookahead {str(y+1)}: {sentence[x+y+1]} ({str(self.reverse_labels[sentence[x+y+1]])})")
                        self.matrix[y][self.reverse_labels[word]][self.reverse_labels[sentence[x+y+1]]] += 1

In [100]:
matrix = HMatrix()
matrix.create_matrix(token_list, 3) #todo, juypter seems to break when order > 3

100%|██████████| 4694/4694 [00:00<00:00, 20552.78it/s]

Word: logitech (2474)
Lookahead 1: g (1890)
Lookahead 2: pro (3047)
Lookahead 3: x (4075)
Word: g (1890)
Lookahead 1: pro (3047)
Lookahead 2: x (4075)
Lookahead 3: superlight (3628)
Word: pro (3047)
Lookahead 1: x (4075)
Lookahead 2: superlight (3628)
Word: x (4075)
Lookahead 1: superlight (3628)
Word: superlight (3628)
Word: nintendo (2757)
Lookahead 1: switch (3649)
Lookahead 2: lite (2460)
Lookahead 3: gray (2039)
Word: switch (3649)
Lookahead 1: lite (2460)
Lookahead 2: gray (2039)
Lookahead 3: with (4057)
Word: lite (2460)
Lookahead 1: gray (2039)
Lookahead 2: with (4057)
Lookahead 3: original (2854)
Word: gray (2039)
Lookahead 1: with (4057)
Lookahead 2: original (2854)
Lookahead 3: box (1264)
Word: with (4057)
Lookahead 1: original (2854)
Lookahead 2: box (1264)
Lookahead 3: extras (1764)
Word: original (2854)
Lookahead 1: box (1264)
Lookahead 2: extras (1764)
Lookahead 3: anbernic (1051)
Word: box (1264)
Lookahead 1: extras (1764)
Lookahead 2: anbernic (1051)
Lookahead 3: rg351

## Sentence Class

Input a sentence, it will then use the HMatrix to see which path results the most likely outcome for products

In [108]:
class Sentence:
    def __init__(self, matrix: HMatrix, tokenizer, raw_sentence: str, verbose=False):
        self.verbose = verbose
        self.raw_sentence = tokenizer.tokenize(raw_sentence.lower())
        self.sentence = []

        # Creating all the blank word classes
        for x, word in enumerate(self.raw_sentence):
            self.sentence.append(Word(word, x, matrix.order))

        # Creating all the forward connections
        for x, word in enumerate(self.raw_sentence):
            if verbose:
                print(f"--- {word} ---")
            for neighbor in self.sentence:
                mapped_pos = self.map_word_pos_to_order(neighbor.position-x)

                if mapped_pos[1] is not direction.SELF and mapped_pos[0] < matrix.order: #Looking at itself and order is within scope
                    p_value = 0

                    if mapped_pos[1] is direction.FORWARD and self.verbose:
                        p_value = matrix.p_row_word(mapped_pos[0], word, neighbor.key)
                    else:
                        p_value = matrix.p_col_word(mapped_pos[0], word, neighbor.key)

                    self.sentence[x].neighbors.append(
                            {'word': neighbor.key,
                             'ref': neighbor,
                             'p': p_value,
                             'direction': mapped_pos[1].name
                            })

                    if verbose:
                        print(f"{'  '*mapped_pos[0]}"
                              f"{word} "
                              f"-{str(mapped_pos[0])}-> "
                              f"{neighbor.key} "
                              f"({str(p_value)})")



    def map_word_pos_to_order(self, position) -> tuple:
        #Forward word positions
        if position > 0:
            return position-1, direction.FORWARD

        # Backwards word positions
        if position < 0:
            return abs(position)-1, direction.BACKWARD

        #Pos looking at self
        if position == 0:
            return -1, direction.SELF



class Word:
    def __init__(self, key: str, position: int, order: int):
        self.key = key
        self.position = position
        self.matrix_order = order
        self.neighbors = []

    # This method will find this word's neighbor by taking the words position + an input position, returns None if out of bounds.
    def get_neighbor(self, pos):
        relative_pos = None
        if pos == 0:
            return None

        if pos > self.matrix_order: #cant look past an order we dont have saved
            return None

        if pos > 0: #positive lookahead
            if self.position >= self.matrix_order:
                if self.matrix_order+(pos-1) > len(self.neighbors)-1:
                    relative_pos = None
                else:
                    relative_pos = self.matrix_order+(pos-1)
            else:
                relative_pos = len(self.neighbors) - self.matrix_order + (pos-1)
        else: #negative lookahead
            print(f"{self.key} | pos: {str(pos)} | absolute pos: {str(self.position)}")
            if self.position < abs(pos):
                relative_pos = None
            else:
                if self.position >= self.matrix_order:
                    relative_pos = self.matrix_order + pos
                else:
                    relative_pos = self.matrix_order*2-len(self.neighbors)-1 #todo this is what we need to modify


        if relative_pos is None:
            return None
        else:
            return self.neighbors[relative_pos]


    def __str__(self):
        return f"Word: {self.key} | Position: {str(self.position)}"

In [109]:
test = Sentence(matrix, tokenizer, "logitech g pro x superlight intel i7 8700k", verbose=False) #logitech g pro x superlight intel i7 8700k

In [110]:
i = 6
test.sentence[i].key,test.sentence[i].position, [x['word'] for x in test.sentence[i].neighbors]

('i7', 6, ['x', 'superlight', 'intel', '8700k'])

In [112]:
i = 1
testdata = test.sentence[i].get_neighbor(-1)
f"{test.sentence[i].key} -> {testdata['word'] if testdata is not None else None}" #todo, order 2 works but order 3 breaks here

g | pos: -1 | absolute pos: 1


'g -> pro'

Next neighbor pos (order 2 matrix):
logitech 0 len(neighbors) - order
g 1 len(neighbors) - order
pro 2 order
x 2 order
superlight 2
intel 2
i7 2
8700k None

if pos >= order:
    if order > len(neighbors)-1:
        return None
    else:
        return order
else:
    return len(neighbors) - order + 0

+2 neighbor pos (order 2 matrix):
logitech 1
g 2
pro 3
x 3
superlight 3
intel 3
i7 None
8700k None

if pos >= order:
    if order + 1 > len(neighbors)-1:
        return None
    else:
        return order + 1
else:
    return len(neighbors) - order + 1


-1 neighbor pos (order 2 matrix):
logitech None
g 0 order*2-len(neighbors)
pro 1 order - 1
x 1 order - 1
superlight 1 order - 1
intel 1 order - 1
i7 1 order - 1
8700k 1 order - 1

if pos < order-1+(0):
    return None
else:
    if relative_pos >= order:
        return order - 1
    else:
        order*2-len(neighbors)-1


-2 neighbor pos (order 2 matrix):
logitech None
g None
pro 0 order -2 + order*2-len(neighbors)
x 0
superlight 0
intel 0
i7 0
8700k 0

if pos < order-1+(1):
    return None
else:
    if relative_pos >= order:
        return order - 2

In [11]:
def seperate_products(start_word: Word):
    products = []



# A method to look ahead only 1 word until the next word probability is 0
def traverse_to_first_order_end(word: Word):
    if word.get_neighbor(1) is None:
        return word

    print(f"Start: {word.key}")
    neighbor = word.get_neighbor(1)

    while neighbor['ref'].get_neighbor(1) is not None and neighbor['ref'].get_neighbor(1)['p'] > 0:
        print(f"Visited: {neighbor['word']}({str(neighbor['p'])})")

        # Go to next neighbor
        neighbor = neighbor['ref'].get_neighbor(1)

    if neighbor['ref'].get_neighbor(1) is None:
        print(f"EoS: {neighbor['word']}") # if at end of sentence
    else:
        print(f"EoP: {neighbor['word']}")# if at end of product

# Take in a sentence, determine if a word should point right or left, then
def vector_assign(sentence: list[Word]):
    v = Vector()
    for x, word in enumerate(sentence):
        print()
        if x == 0: #First word always points right
            v.add(word.key, "r")
            print(f"{word.key} -> {sentence[x].get_neighbor(1)['word']}")
        elif x == len(sentence)-1: #end of sentence
            v.add(word.key, "l")
            print(f"{sentence[x].get_neighbor(-1)['word']} <- {word.key}")
        else:
            if sentence[x].get_neighbor(1)['p'] >= sentence[x].get_neighbor(-1)['p']:
                v.add(word.key, "r")
                print(f"{word.key} -> {sentence[x].get_neighbor(1)['word']}")
            else:
                v.add(word.key, "l")
                print(f"{sentence[x].get_neighbor(-1)['word']} <- {word.key}")
    return v


In [12]:
class Vector:
    def __init__(self):
        self.items = []

    def add(self, item, vector):
        if len(self.items) == 0:
            self.items.append({'items': [item], 'current_vector': vector})
        else:
            if self.items[-1]['current_vector'] == "r": #if current vector is right we dont care about the direction
                self.items[-1]['items'].append(item)
                self.items[-1]['current_vector'] = vector
            elif self.items[-1]['current_vector'] == "l":
                if vector == 'r': # new item
                    self.items.append({'items': [item], 'current_vector': vector})
                elif vector == 'l': # old item
                    self.items[-1]['items'].append(item)
                    self.items[-1]['current_vector'] = vector

In [13]:
vector_assign(test.sentence).items


logitech: 0
logitech -> g

g: 1
g: 0
g: 0
logitech <- g

pro: 2
pro: 1
pro: 1
g <- pro

x: 3
x: 2
x: 2
superlight <- x



TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Stress Testing The Vector method among all words
count = 0
for word in token_list[100:200]:
    s = Sentence(matrix, tokenizer, " ".join(word), verbose=False)
    print("---")
    print(count)
    print(" ".join(word))
    print(vector_assign(s.sentence).items)
    print("---\n")
    count += 1


In [ ]:
vector_assign(test.sentence)

In [ ]:
traverse_to_first_order_end(test.sentence[0])

From word a to word b (order 0) there are the following outcomes:
* p(a,b) > 0.0 has 2 outcomes
    * b is part of a
    * b is NOT part of a, but has been seen in the training data
* p(a,b) = 0.0 has 2 outcomes
    * b is NOT part of a
    * b is a new word
        * b is part of a
        * b is part of post b

In [ ]:
matrix.p_row_word(0, 'rtx', '5080')

I believe the case above where 5080 is a new word, is where we want to look at the word 'rtx' and see what is the probability that it is the final word. If it's close to 0, then we should determine that 5080 is tied to it

In [ ]:
test.sentence[0].neighbors

current thoughts, look at a sentence, for each word choose to point at either right or left, add up all the points and those will be your products